# Extraction Notebook

In [2]:
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

## Testing and model config

In [4]:
chat = ChatAnthropic(temperature=0, model_name="claude-3-opus-20240229")

In [5]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate

chat = ChatAnthropic(temperature=0, model_name="claude-3-opus-20240229")

system = (
    "You are a helpful assistant that translates {input_language} to {output_language}."
)
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke(
    {
        "input_language": "English",
        "output_language": "Korean",
        "text": "I love Python",
    }
)

AIMessage(content='저는 파이썬을 사랑합니다.\n\nTranslation:\nI love Python.', response_metadata={'id': 'msg_01YQxnA6usGmYdERQS12DZc7', 'model': 'claude-3-opus-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 22, 'output_tokens': 28}}, id='run-5b715f5c-189c-4222-95f1-6a6ba9bf67ec-0')

## Extraction Chain

In [21]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

In [7]:
from typing import List, Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person. This field is required.")
    image_url: Optional[str] = Field('', description="URL of an image of the person. This is usually left empty.")
    facts: List[str] = Field(default_factory=list, description="A list of miscellaneous facts about the person.")

In [17]:
text = """ Speaker A:  Hey man, how's it going? 

Speaker B:  Good, good. How about you? What's your favorite animal, man? 

Speaker A:  Pretty good. My favorite animal is dinosaur. But let me start, like, what's your name? 

Speaker B:  I'm Arush. What's yours? 

Speaker A:  I'm Vig, by the way. Nice to meet you. So how do you like the conference? 

Speaker B:  It's pretty cool. It's really big. I'm meeting a lot of really cool people. 

Speaker A:  My favorite animal is the capybara, so I've been finding a lot of capybara lovers around here. It's been nice. 

Speaker B:  Tell me more about capybara. 

Speaker A:  They're just really goofy. 

Speaker B:  Interesting. Is that why you like them? 

Speaker A:  Yeah, yeah. It's... 

Speaker B:  That's very interesting, man. Like, I've never seen someone who's a fan of capybara. 

Speaker A:  It's the first time. 

Speaker B:  Yeah, they're like huge rats, basically. 

Speaker A:  Oh, okay. Interesting. 

Speaker B:  They're really cute, and there's been a trend going on, going around on, like, Instagram and stuff of 

Speaker A:  reels of capybaras doing random stuff. They're like pandas, by the way. 

Speaker B:  That's amazing. It kind of reminds me of how, you know, people use animal names for, like, technology stuff. 

Speaker A:  Like, you know, I don't want to quite say LLM, but, you know, llama is something. 

Speaker B:  Yeah. 

Speaker A:  So, but that's very interesting. I'm a huge fan of dinosaur, and one of the popular things is just, I think they're very cool. 

Speaker B:  I used to be a very much a fan of a velociraptor, which is like, you know, a dinosaur, right? 

Speaker A:  Yeah. What really sparked your interest in velociraptors? Was it Jurassic Park? 

Speaker B:  I could, it's partly the Jurassic Park, but I would say it's more like the way they behave. 

Speaker A:  Something that was very attractive is just like, you know, they're very intelligent. 

Speaker B:  They work as a group, and, you know, they're kind of excited, like, it's kind of like they're not very big, 

Speaker A:  but they're like super genius, right? 

Speaker B:  Yeah. 

Speaker A:  Like the dog, yes, but, all right. I hope you have a good conference, man. 

Speaker B:  Yeah, you too, man. 

Speaker A:  I'll see you around. See ya.
"""

In [18]:
# runnable = prompt | chat.with_structured_output(schema=Person)

# runnable.invoke({"text": text, "input_language": "English",
#         "output_language": "English",})


In [22]:
class Conversation(BaseModel):
    """Model for capturing conversation details.

    This class captures the essentials of a conversation, including who was involved, when it occurred, and a summary of the discussion. The timestamp is optional and can be left empty, aligning with the requirement to occasionally omit the specific time of the conversation.
    """

    persons: List[Person]
    timestamp: Optional[str] = Field(None, description="The timestamp when the conversation occurred. This field is always empty.")
    summary:Optional[str] = Field(..., description="A bulleted summary list of key points discussed in the conversation.")

In [23]:
runnable = prompt | chat.with_structured_output(schema=Conversation)

runnable.invoke({"text": text})


Conversation(persons=[Person(name='Vig', image_url='', facts=["Vig's favorite animal is the capybara", 'Vig likes capybaras because they are goofy', 'Vig has been finding other capybara lovers at the conference']), Person(name='Arush', image_url='', facts=['Arush used to be a fan of velociraptors', 'Arush was partly interested in velociraptors due to Jurassic Park', 'Arush likes how velociraptors behave intelligently in groups despite their small size'])], timestamp='', summary='- Vig and Arush introduce themselves to each other at a conference\n- Vig shares that his favorite animal is the capybara because they are goofy\n- Vig has been meeting other capybara fans at the conference \n- Arush used to be a fan of velociraptors, partly due to Jurassic Park\n- Arush likes how velociraptors work intelligently in groups despite being small\n- They discuss the trend of capybara videos on social media\n- They note how animal names are sometimes used for technology \n- They wish each other well